In [ ]:
import ijson
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
rows = []

# Only for permit issuance data as using read_json does not read the JSON file properly
# So, doing it this way
with open("ipu4-2q9a - DOB Permit Issuance.json", 'rb') as f:
    for obj in ijson.items(f, 'item'):
        rows.append(obj)

permits = pd.DataFrame(rows)
restaurants = pd.read_json('43nn-pn8j - NYC Restaurant Inspection Results.json')  # restaurant_name, lat, lon, inspection_date
rat_complaints = pd.read_csv('Rat_Sightings_20250428.csv')  # lat, lon, complaint_date

In [ ]:
# Drop NaNs
restaurants = restaurants.dropna(subset=['dba', 'latitude', 'longitude', 'inspection_date'])
rat_complaints = rat_complaints.dropna(subset=['Latitude', 'Longitude', 'Created Date'])
permits = permits.dropna(subset=['gis_latitude', 'gis_longitude', 'job_start_date', 'expiration_date'])

# Convert dates
restaurants['inspection_date'] = pd.to_datetime(restaurants['inspection_date'],errors = 'coerce')
rat_complaints['complaint_date'] = pd.to_datetime(rat_complaints['Created Date'],errors = 'coerce')
permits['permit_start_date'] = pd.to_datetime(permits['job_start_date'],errors = 'coerce')
permits['permit_end_date'] = pd.to_datetime(permits['expiration_date'],errors = 'coerce')

permits['gis_latitude'] = permits['gis_latitude'].astype(float)
permits['gis_longitude'] = permits['gis_longitude'].astype(float)


# Extract month period
restaurants['inspection_month'] = restaurants['inspection_date'].dt.to_period('M')
rat_complaints['complaint_month'] = rat_complaints['complaint_date'].dt.to_period('M')

# Convert to radians
def to_radians(df, lat_col, lon_col):
    df['lat_rad'] = np.radians(df[lat_col])
    df['lon_rad'] = np.radians(df[lon_col])
    return df

restaurants = to_radians(restaurants, 'latitude', 'longitude')
rat_complaints = to_radians(rat_complaints, 'Latitude', 'Longitude')
permits = to_radians(permits, 'gis_latitude', 'gis_longitude')

# Build BallTrees
rat_tree = BallTree(rat_complaints[['lat_rad', 'lon_rad']], metric='haversine')
permit_tree = BallTree(permits[['lat_rad', 'lon_rad']], metric='haversine')

# Radius in radians (0.5 miles)
# Currently same for both rat and construction data
# We can change it to reflect each one differently
radius = 0.5 / 3956

In [ ]:
results = []

for _, row in restaurants.iterrows():
    rest_point = np.array([[row['lat_rad'], row['lon_rad']]])
    
    # Rat complaints in radius and same month
    rat_indices = rat_tree.query_radius(rest_point, r=radius)[0]
    complaint_count = 0
    if len(rat_indices) > 0:
        nearby_complaints = rat_complaints.iloc[rat_indices]
        same_month_complaints = nearby_complaints[
            nearby_complaints['complaint_month'] == row['inspection_month']
        ]
        complaint_count = len(same_month_complaints)

    # Permits in radius and inspection_date in [start, end]
    permit_indices = permit_tree.query_radius(rest_point, r=radius)[0]
    construction_count = 0
    if len(permit_indices) > 0:
        nearby_permits = permits.iloc[permit_indices]
        active_construction = nearby_permits[
            (nearby_permits['permit_start_date'] <= row['inspection_date']) &
            (nearby_permits['permit_end_date'] >= row['inspection_date'])
        ]
        construction_count = len(active_construction)

    results.append({
        'restaurant_name': row['dba'],
        'inspection_month': str(row['inspection_month']),
        'lat': row['latitude'],
        'lon': row['longitude'],
        'rat_complaint_count': complaint_count,
        'construction_count': construction_count
    })


final_df = pd.DataFrame(results)
print(final_df.head())


In [ ]:
# Saving the dataframe as CSV file
final_df.to_csv(f"restaurants 0.5mi radius rodent and construction count.csv", index=False, header=True)